# Re-create your own _One Hot Encoder_ 

In [1]:
import pandas as pd
import seaborn as sns

## (1) The Titanic Dataset

In [2]:
data = sns.load_dataset('titanic').sample(frac = 1) 

In [3]:
from sklearn.model_selection import train_test_split

X = data.drop(columns = ['survived', 'alive', 'who', 'adult_male'])
y = data['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [4]:
X_train

## (2) A first pipeline

❓ Create a basic Pipeline which ***encodes categorical features*** and ***scales numerical features*** ❓

In [5]:
num_features = ['age','fare','sibsp','parch']
cat_features = ['pclass','sex','embarked','class','embark_town','alone']

In [6]:
# YOUR CODE HERE

❓ Chain this preprocessing pipeline with a classifier and optimize it ❓

In [8]:
# YOUR CODE HERE

❓ What are the best params and the best score ❓

In [9]:
# YOUR CODE HERE

## (3) How could we design a Custom Encoder to keep track of the columns' names?

In [11]:
# By default, OneHotEncoder works with Numpy and loses track of columns' names...
ohe = OneHotEncoder(sparse=False)
ohe.fit_transform(X_train[['sex']])

In [12]:
# ... however, we can access the one-hot-encoded names as follows
ohe.get_feature_names_out()

❓ Try to create your own OneHotEncoder so that it preserves the columns names ❓

In [ ]:
# YOUR CODE HERE

🏁 If you want to build a very advanced pipeline, feel free to explore the Optional Challenge dealing the `cars dataset` !

💾 Don't forget to git add/commit/push your notebook.

👏 Congratulations, you are now a master at Pipeline and ColumnTransformer.